In [12]:
pip install dash plotly

In [16]:
pip install dash dash-bootstrap-components pandas plotly

In [18]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc

# URL do arquivo CSV no GitHub (raw link ajustado)
url = "https://raw.githubusercontent.com/VitorFerreiraVHF/ATV_Final-Python/main/Dados/vendas.csv"

# Carregar dados
df = pd.read_csv(url)

# Ajustar colunas
df['Data da Venda'] = pd.to_datetime(df['Data da Venda'])  # Converter para datetime

# Função para criar os KPI Cards
def generate_card(header, value, color):
    return dbc.Card(
        dbc.CardBody([
            html.H5(header, className="card-title text-center"),
            html.H4(value, className="card-text text-center"),
        ]),
        color=color, inverse=True, className="mb-3"
    )

# Criar a aplicação Dash com tema Bootstrap
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Dashboard Logístico"

# Layout do aplicativo
app.layout = dbc.Container(
    fluid=True,
    children=[
        # Cabeçalho
        dbc.Row(
            dbc.Col(
                html.H1("Dashboard Logístico", className="text-center text-primary mb-4"),
                width=12
            )
        ),
        # KPIs em Cards
        dbc.Row([
            dbc.Col(generate_card("Total de Vendas", df["Qtd Vendida"].sum(), "primary"), width=3),
            dbc.Col(generate_card("Total de Ordens de Compra", df["Ordem de Compra"].nunique(), "info"), width=3),
            dbc.Col(generate_card("Total de Clientes", df["ID Cliente"].nunique(), "success"), width=3),
            dbc.Col(generate_card("Lojas Ativas", df["ID Loja"].nunique(), "warning"), width=3),
        ]),
        # Filtro e Gráficos
        dbc.Row([
            # Dropdown de loja
            dbc.Col(
                dcc.Dropdown(
                    id="dropdown_loja",
                    options=[{"label": loja, "value": loja} for loja in df["ID Loja"].unique()],
                    value=df["ID Loja"].iloc[0],
                    placeholder="Selecione uma Loja",
                    style={"margin-bottom": "20px"}
                ), width=4
            ),
        ]),
        # Gráficos em duas colunas
        dbc.Row([
            dbc.Col(dcc.Graph(id="grafico_linhas"), width=6),
            dbc.Col(dcc.Graph(id="grafico_barras"), width=6),
        ]),
        dbc.Row([
            dbc.Col(dcc.Graph(id="grafico_pizza"), width=6),
            dbc.Col(dcc.Graph(id="grafico_dispersao"), width=6),
        ]),
        # Histograma e Tabela
        dbc.Row([
            dbc.Col(dcc.Graph(id="grafico_histograma"), width=6),
            dbc.Col(dcc.Graph(id="tabela_resumo"), width=6),
        ]),
    ],
    style={"margin": "20px"}  # Margem geral para o layout
)

# Callback para atualizar gráficos dinamicamente
@app.callback(
    [
        Output("grafico_linhas", "figure"),
        Output("grafico_barras", "figure"),
        Output("grafico_pizza", "figure"),
        Output("grafico_dispersao", "figure"),
        Output("tabela_resumo", "figure"),
        Output("grafico_histograma", "figure")
    ],
    [Input("dropdown_loja", "value")]
)
def atualizar_graficos(loja_selecionada):
    # Filtrar dados pela loja selecionada
    df_filtrado = df[df["ID Loja"] == loja_selecionada]

    # Gráfico de Linhas: Vendas ao longo do tempo
    fig_linhas = px.line(
        df_filtrado.groupby("Data da Venda")["Qtd Vendida"].sum().reset_index(),
        x="Data da Venda", y="Qtd Vendida", title="Vendas Totais ao Longo do Tempo"
    )

    # Gráfico de Barras: Vendas por Loja
    fig_barras = px.bar(
        df.groupby("ID Loja")["Qtd Vendida"].sum().reset_index(),
        x="ID Loja", y="Qtd Vendida", title="Vendas por Loja"
    )

    # Gráfico de Pizza: Proporção de Vendas por SKU
    fig_pizza = px.pie(
        df_filtrado.groupby("SKU")["Qtd Vendida"].sum().reset_index(),
        values="Qtd Vendida", names="SKU", title="Proporção de Vendas por SKU"
    )

    # Gráfico de Dispersão: Relação entre SKU e Quantidade Vendida
    fig_dispersao = px.scatter(
        df_filtrado, x="Ordem de Compra", y="Qtd Vendida", color="SKU",
        title="Relação entre Ordens de Compra e Quantidade Vendida"
    )

    # Tabela Dinâmica: Resumo por Loja
    tabela = df.groupby("ID Loja")[["Qtd Vendida"]].sum().reset_index()
    fig_tabela = px.bar(tabela, x="ID Loja", y="Qtd Vendida", title="Resumo de Vendas por Loja")

    # Histograma: Distribuição de Quantidade Vendida
    fig_histograma = px.histogram(
        df_filtrado, x="Qtd Vendida", nbins=10, title="Distribuição de Quantidades Vendidas"
    )

    return fig_linhas, fig_barras, fig_pizza, fig_dispersao, fig_tabela, fig_histograma


# Rodar o servidor
if __name__ == "__main__":
    app.run_server(debug=True)


<IPython.core.display.Javascript object>